In [157]:
# imports 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split


from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression


from sklearn.metrics import mean_squared_error


In [158]:
df= pd.read_csv('../data/train/diamonds_train.csv', index_col=[0])
df

,index_id,depth,table,x,y,z,price,carat,cut,color,clarity,city
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,62.4,58.0,6.83,6.79,4.25,4268,1.21,Premium,J,VS2,Dubai
1,41667f6e2629360aecaf00b20f8732e3310417ebd54b24...,61.6,58.0,6.40,6.35,3.93,3513,1.02,Premium,J,VS2,Dubai
2,01f8667f50d52677bea23231a74156e4f92360d7bc3db6...,62.3,58.0,5.86,5.80,3.63,1792,0.77,Premium,J,VS2,Dubai
3,c3867352aab641358faec75d733af012dbe2259a014ea8...,59.6,60.0,7.58,7.48,4.49,7553,1.51,Premium,J,VS2,Dubai
4,0da4b104c4d8589fcb96a03aa0787549a2631935b0f499...,60.2,62.0,5.40,5.33,3.23,1176,0.57,Premium,J,VS2,Dubai
...,...,...,...,...,...,...,...,...,...,...,...,...
40450,3df6f3da962b819057888dbbe3cf4d11f9a59261ba0af7...,62.2,54.0,5.24,5.27,3.27,2729,0.54,Ideal,F,IF,Surat
40451,e5bc95d36abda5bfd67641eca60f2ab92f37b27c0397e0...,61.9,54.0,5.22,5.25,3.24,2802,0.53,Ideal,F,IF,Surat
40452,5ad0fcca0db9b3d399e31a3d909a1a3f4da38c663256b3...,62.3,55.0,4.30,4.34,2.69,886,0.30,Ideal,F,IF,Surat
40453,c8559278b1ac356e1e414320a4efbbe788dc16623e7873...,60.9,55.0,4.15,4.23,2.55,768,0.26,Ideal,F,IF,Surat


In [159]:
df.drop(labels='index_id', axis=1, inplace= True)

In [160]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40455 entries, 0 to 40454
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   depth    40455 non-null  float64
 1   table    40455 non-null  float64
 2   x        40455 non-null  float64
 3   y        40455 non-null  float64
 4   z        40455 non-null  float64
 5   price    40455 non-null  int64  
 6   carat    40455 non-null  float64
 7   cut      40455 non-null  object 
 8   color    40455 non-null  object 
 9   clarity  40455 non-null  object 
 10  city     40455 non-null  object 
dtypes: float64(6), int64(1), object(4)
memory usage: 3.7+ MB


In [161]:
# categorical features

cat_cols=['cut', 'color','clarity', 'city']

cat_list = []
for col in cat_cols:
    cat = df[col].unique()
    cat_num = len(cat)
    cat_dict = {"categorical_variable":col,
                "number_of_possible_values":cat_num,
                "values":cat}
    cat_list.append(cat_dict)
    
categories = pd.DataFrame(cat_list).sort_values(by="number_of_possible_values",
                                                ascending=False).reset_index(drop=True)
categories

,categorical_variable,number_of_possible_values,values
0,city,13,"[Dubai, Luxembourg, New York City, Antwerp, Ma..."
1,clarity,8,"[VS2, VVS2, SI1, VS1, SI2, I1, VVS1, IF]"
2,color,7,"[J, E, I, G, D, H, F]"
3,cut,5,"[Premium, Very Good, Fair, Good, Ideal]"


Por el momento, no vamos a tener en cuenta la columna de la ciudad, ya que hay demasiados valores.

In [162]:
df_one_hot_encoding = pd.get_dummies(df, 
                                    columns=['cut', 'color','clarity'], 
                                    drop_first=True)
df_one_hot_encoding.drop(labels='city', axis=1, inplace=True)
df_one_hot_encoding

,depth,table,x,y,z,price,carat,cut_Good,cut_Ideal,cut_Premium,...,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,62.4,58.0,6.83,6.79,4.25,4268,1.21,0,0,1,...,0,0,1,0,0,0,0,1,0,0
1,61.6,58.0,6.40,6.35,3.93,3513,1.02,0,0,1,...,0,0,1,0,0,0,0,1,0,0
2,62.3,58.0,5.86,5.80,3.63,1792,0.77,0,0,1,...,0,0,1,0,0,0,0,1,0,0
3,59.6,60.0,7.58,7.48,4.49,7553,1.51,0,0,1,...,0,0,1,0,0,0,0,1,0,0
4,60.2,62.0,5.40,5.33,3.23,1176,0.57,0,0,1,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,62.2,54.0,5.24,5.27,3.27,2729,0.54,0,1,0,...,0,0,0,1,0,0,0,0,0,0
40451,61.9,54.0,5.22,5.25,3.24,2802,0.53,0,1,0,...,0,0,0,1,0,0,0,0,0,0
40452,62.3,55.0,4.30,4.34,2.69,886,0.30,0,1,0,...,0,0,0,1,0,0,0,0,0,0
40453,60.9,55.0,4.15,4.23,2.55,768,0.26,0,1,0,...,0,0,0,1,0,0,0,0,0,0


In [163]:
df_one_hot_encoding.columns

Index(['depth', 'table', 'x', 'y', 'z', 'price', 'carat', 'cut_Good',
       'cut_Ideal', 'cut_Premium', 'cut_Very Good', 'color_E', 'color_F',
       'color_G', 'color_H', 'color_I', 'color_J', 'clarity_IF', 'clarity_SI1',
       'clarity_SI2', 'clarity_VS1', 'clarity_VS2', 'clarity_VVS1',
       'clarity_VVS2'],
      dtype='object')

In [187]:
x_cols=['depth', 'table', 'x', 'y', 'z', 'carat', 'cut_Good',
       'cut_Ideal', 'cut_Premium', 'cut_Very Good', 'color_E', 'color_F',
       'color_G', 'color_H', 'color_I', 'color_J', 'clarity_IF', 'clarity_SI1',
       'clarity_SI2', 'clarity_VS1', 'clarity_VS2', 'clarity_VVS1',
       'clarity_VVS2']
X= df_one_hot_encoding[x_cols]
y= df_one_hot_encoding['price']
print(X.shape, y.shape)

(40455, 23) (40455,)


In [175]:
# Train and test datasets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}")


X_train: (34386, 23), X_test: (6069, 23), y_train: (34386,), y_test: (6069,)


In [176]:
# Model definition

model = linear_model.Lasso()
#model = ElasticNet()
#model = Ridge()
#model = SVR()
#model = SGDRegressor()
#model = LinearRegression()



In [177]:
# Model training
model.fit(X_train, y_train)


Lasso()

In [178]:
# Model predictions

y_pred = model.predict(X_test)
y_pred

array([7245.54083048,  884.50620741, 8519.87035027, ..., 1265.01505976,
       1118.88771002, 2070.68332175])

In [179]:
# Visual check

rmse = mean_squared_error(y_test, y_pred)**0.5
rmse

1175.3939504123528

In [180]:
#Model prediction Test Dataframe (aka real prediction):
diamonds_train=pd.read_csv('../data/test/diamonds_test.csv', index_col=[0])
diamonds_train

,carat,cut,color,clarity,depth,table,x,y,z,city
id,,,,,,,,,,
0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam
...,...,...,...,...,...,...,...,...,...,...
13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam
13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City
13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv


In [181]:
diamonds_train_encoded= pd.get_dummies(diamonds_train, 
                                    columns=['cut', 'color','clarity'], 
                                    drop_first=True)
diamonds_train_encoded.drop(labels='city', axis=1, inplace=True)
diamonds_train_encoded

,carat,depth,table,x,y,z,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,...,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
id,,,,,,,,,,,,,,,,,,,,,
0,0.79,62.7,60.0,5.82,5.89,3.67,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
1,1.20,61.0,57.0,6.81,6.89,4.18,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
2,1.57,62.2,61.0,7.38,7.32,4.57,0,0,1,0,...,1,0,0,0,1,0,0,0,0,0
3,0.90,63.8,54.0,6.09,6.13,3.90,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
4,0.50,62.9,58.0,5.05,5.09,3.19,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0.57,61.9,56.0,5.35,5.32,3.30,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
13481,0.71,62.2,55.0,5.71,5.73,3.56,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0
13482,0.70,61.6,55.0,5.75,5.71,3.53,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0


In [182]:
#We must order the columns as per the train dataframe columns order:
diamonds_train_encoded=diamonds_train_encoded[['depth', 'table', 'x', 'y', 'z', 'carat', 'cut_Good', 'cut_Ideal',
       'cut_Premium', 'cut_Very Good', 'color_E', 'color_F', 'color_G',
       'color_H', 'color_I', 'color_J', 'clarity_IF', 'clarity_SI1',
       'clarity_SI2', 'clarity_VS1', 'clarity_VS2', 'clarity_VVS1',
       'clarity_VVS2']]

In [188]:
model.fit(X,y) #Volvemos a entrenar el modelo con el datframe original (sin la separacion de train y test), para tener mas datos en el entreno

Lasso()

In [190]:
y_pred = model.predict(diamonds_train_encoded)
y_pred=pd.DataFrame(y_pred, columns=['price'])
y_pred.reset_index(inplace=True)
y_pred.rename(columns={"index": "id"}, inplace= True)
y_pred


,id,price
0,0,3603.709526
1,1,6327.398656
2,2,10071.375009
3,3,4663.696418
4,4,2084.625114
...,...,...
13480,13480,1968.069679
13481,13481,2549.744413
13482,13482,3902.344316
13483,13483,2003.748570


In [191]:
y_pred.to_csv('../data/prediction/price_prediction.csv', index= False)